<a href="https://colab.research.google.com/github/LeoMcBills/tensorFlowPrac/blob/main/legendZoro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import necessary pytorch modules

In [94]:
import torch
from torch import nn

# Download text-tinyShakespeare

In [95]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-03-21 14:34:44--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2024-03-21 14:34:45 (20.0 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [96]:
!ls

input.txt  input.txt.1	sample_data


In [97]:
!rm input.txt.1

In [98]:
with open('input.txt', 'r', encoding='utf-8') as text:
  text = text.read()

In [99]:
print(len(text))
print(text[:1000])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for re

In [100]:
chars = sorted(list(set(text)))
print(len(chars))
print("".join(chars))

65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [101]:
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
vocab_size = len(chars)

In [102]:
print(vocab_size)
print(stoi)
print(itos)

65
{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: '

In [103]:
# method 1
def tokenize(word):
  newrd = []
  for piece in word:
    newrd.append(stoi[piece])
  return newrd

In [104]:
# method 2
encode = lambda wrd: [stoi[char] for char in wrd]

In [105]:
encoded = tokenize("Leo is the greatest programmer living")
encoded2 = encode("Leo is the greatest programmer living")

In [106]:
print(encoded)
print(encoded2)

[24, 43, 53, 1, 47, 57, 1, 58, 46, 43, 1, 45, 56, 43, 39, 58, 43, 57, 58, 1, 54, 56, 53, 45, 56, 39, 51, 51, 43, 56, 1, 50, 47, 60, 47, 52, 45]
[24, 43, 53, 1, 47, 57, 1, 58, 46, 43, 1, 45, 56, 43, 39, 58, 43, 57, 58, 1, 54, 56, 53, 45, 56, 39, 51, 51, 43, 56, 1, 50, 47, 60, 47, 52, 45]


In [107]:
# method 1
def wordFromToken(tokens):
  sentence = ""
  for token in tokens:
    sentence += itos[token]
  return sentence

In [108]:
print(wordFromToken(encoded))

Leo is the greatest programmer living


In [109]:
# method 2
decode = lambda wrd: [itos[num] for num in wrd]

In [110]:
print("".join(decode(encoded)))

Leo is the greatest programmer living


# Get the training and test data

In [131]:
train_set = tokenize(text[:int(len(text)*0.9)])
test_set = tokenize(text[int(len(text)*0.9):])

In [132]:
print(train_set[:1000])
print(test_set[:1000])

[18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 14, 43, 44, 53, 56, 43, 1, 61, 43, 1, 54, 56, 53, 41, 43, 43, 42, 1, 39, 52, 63, 1, 44, 59, 56, 58, 46, 43, 56, 6, 1, 46, 43, 39, 56, 1, 51, 43, 1, 57, 54, 43, 39, 49, 8, 0, 0, 13, 50, 50, 10, 0, 31, 54, 43, 39, 49, 6, 1, 57, 54, 43, 39, 49, 8, 0, 0, 18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 37, 53, 59, 1, 39, 56, 43, 1, 39, 50, 50, 1, 56, 43, 57, 53, 50, 60, 43, 42, 1, 56, 39, 58, 46, 43, 56, 1, 58, 53, 1, 42, 47, 43, 1, 58, 46, 39, 52, 1, 58, 53, 1, 44, 39, 51, 47, 57, 46, 12, 0, 0, 13, 50, 50, 10, 0, 30, 43, 57, 53, 50, 60, 43, 42, 8, 1, 56, 43, 57, 53, 50, 60, 43, 42, 8, 0, 0, 18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 18, 47, 56, 57, 58, 6, 1, 63, 53, 59, 1, 49, 52, 53, 61, 1, 15, 39, 47, 59, 57, 1, 25, 39, 56, 41, 47, 59, 57, 1, 47, 57, 1, 41, 46, 47, 43, 44, 1, 43, 52, 43, 51, 63, 1, 58, 53, 1, 58, 46, 43, 1, 54, 43, 53, 54, 50, 43, 8, 0, 0, 13, 50, 50, 10, 0, 35, 43, 1, 49, 52, 53, 61, 5, 

## create input tensors

In [133]:
train_set = torch.tensor(train_set, dtype=torch.long)
test_set = torch.tensor(test_set, dtype=torch.long)

In [134]:
print(train_set)
print(test_set)

tensor([18, 47, 56,  ..., 43, 56, 43])
tensor([12,  0,  0,  ..., 45,  8,  0])


In [115]:
batch_size = 4
block_size = 8

In [135]:
print(train_set[:block_size+1])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])


In [136]:
# get x and y
x = train_set[:block_size]
y = train_set[1:block_size+1]
for i in x:
  for j in y:
    print(f"When input is: {i} the target is {j}")
    i = j
  break

When input is: 18 the target is 47
When input is: 47 the target is 56
When input is: 56 the target is 57
When input is: 57 the target is 58
When input is: 58 the target is 1
When input is: 1 the target is 15
When input is: 15 the target is 47
When input is: 47 the target is 58


In [137]:
# Style by Andrej Karpathy
x = train_set[:block_size]
y = train_set[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [118]:
print(x)
print(y)

tensor([18, 47, 56, 57, 58,  1, 15, 47])
tensor([47, 56, 57, 58,  1, 15, 47, 58])


In [147]:
torch.manual_seed(1337)
batch_size = 4  # Number of independent sequences to be processed in parallel
block_size = 8  # Maximum context length for predictions

def get_batch(split):
  # Point to do batching is to generate small batch of data of inputs x and targets y
  data = train_set if split == 'train' else test_set
  ix = torch.randint(len(data) - batch_size, (batch_size,))
  x = torch.stack([data[i+1:i+block_size+1] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print()
print('*************************************************************')
for b in range(batch_size): # Given as batch_dimension
  for t in range(block_size): # time dimension
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"When input is {context.tolist()} the target: {target}")
print('*************************************************************')

inputs:
torch.Size([4, 8])
tensor([[60, 39, 47, 50,  1, 63, 53, 59],
        [43, 39, 60, 43, 52,  1, 44, 53],
        [46, 43, 56, 43,  1, 63, 53, 59],
        [47, 50, 50,  1, 57, 39, 63,  0]])
targets:
torch.Size([4, 8])
tensor([[60, 39, 47, 50,  1, 63, 53, 59],
        [43, 39, 60, 43, 52,  1, 44, 53],
        [46, 43, 56, 43,  1, 63, 53, 59],
        [47, 50, 50,  1, 57, 39, 63,  0]])

*************************************************************
When input is [60] the target: 60
When input is [60, 39] the target: 39
When input is [60, 39, 47] the target: 47
When input is [60, 39, 47, 50] the target: 50
When input is [60, 39, 47, 50, 1] the target: 1
When input is [60, 39, 47, 50, 1, 63] the target: 63
When input is [60, 39, 47, 50, 1, 63, 53] the target: 53
When input is [60, 39, 47, 50, 1, 63, 53, 59] the target: 59
When input is [43] the target: 43
When input is [43, 39] the target: 39
When input is [43, 39, 60] the target: 60
When input is [43, 39, 60, 43] the target: 43
When 

# Build a simple Bigram model

In [119]:
class NeuralNetBigram(nn.Module):
  pass